# Imports & Settings

In [1]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import re 
from math import isnan

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [2]:
# dtype={'type': str} prevents being confused with data type for large data sets
train = pd.read_csv('data/train.csv', index_col='id', dtype={'type': str})
test = pd.read_csv('data/test.csv', index_col='id', dtype={'type': str})
train_translated = pd.read_csv('data/train_translated.csv', dtype={'type': str})
test_translated = pd.read_csv('data/test_translated.csv', index_col='id', dtype={'type': str})
combined_data = pd.read_csv('data/combined_data.csv', index_col='id', dtype={'type': str})
combined_data_translated = pd.read_csv('data/combined_data_translated.csv', index_col='id', dtype={'type': str})
combined_data_fully_translated = pd.read_csv('data/combined_data_fully_translated.csv', index_col='id', dtype={'type': str})
prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})

C:\Users\daria\AppData\Local\Temp\ipykernel_6720\1699356385.py:9: DtypeWarning: Columns (7,253) have mixed types. Specify dtype option on import or set low_memory=False.
  prep = pd.read_csv('data/prep.csv', index_col='id', dtype={'type': str})


# Understanding the attributes

The following columns need to be Hot Encoded:
- musealia_additional_nr -> Threshold = 10 or even 20?
- collection_mark 
- state
- colour -> grouped into base colour groups first 

Instead of mapping 0/1
- museum_abbr
- class
- is_orginal

Extra
- country - Threshold around 10? or 20?
- city_municipality - Threshold around 10?  or 20?
- event_type - Threshold around 30?
- location - expanded into location_city', 'location_building', 'location_street', 'location_country', 'location_address'
- before_Christ - rename ei to no 
- country_and_unit - already adapted by till ?
- participants_role -> Threshold around 
- parish -> Threshold around 3? - v.sparse! almost not worth including
- collection_additional_nr -> Threshold 100 - only letters
- damages -> Threshold 16?


Mapping -> dose value exist yes/no 
- participant -> Threshold 100?
- musealia_mark? -> simple hot_encode - only has 1 value 

In [3]:
data = prep.copy()

In [4]:
data.head()

,ks,event_type,location,before_Christ,participants_role,participant,parish,class,museum_abbr,musealia_mark,musealia_seria_nr,musealia_queue_nr,musealia_additional_nr,collection_mark,collection_queue_nr,collection_additional_nr,element_count,is_original,damages,state,color,type,source,city_municipality,country,material_Polish,material_RC Photo Paper,material_a pearl,material_acetate cellulose film,material_albumen paper,material_albumin paper,material_aluminium,material_amber,material_artificial fiber material,material_artificial leather,material_artificial material,material_atlas,material_ballpoint pen ink,material_birch,material_bone,material_brass,material_brocade (clothing variety),material_bronze,material_canvas,material_canvas (type of cloth),material_cardboard,material_cast iron,material_celluloid,material_ceramics,material_chalk,material_chamois leather,material_chamotte,material_charcoal,material_chromogen emulsion,material_chromogen paper,material_clay,material_clothing variety,material_collodion paper,material_colloid paper,material_colour,material_copper,material_cotton,material_crepe,material_crepe (cloth type),material_crystal,material_diffusion paper,material_email,material_emulsion,material_enamel paint,material_faience,material_feather,material_film,material_film (material),material_flint,material_from the bat,material_glass,material_gold,material_granite,material_graphite,material_gypsum,material_handmade paper,material_ink,material_iron,material_kalka,material_knitwear,material_leotard (type of clothing),material_linen,material_mascara,material_metal,material_metal fibers,material_moire (clothing variety),material_movie,material_nan,material_newsprint,material_nitrocellulose film,material_nut,material_oil paint,material_organic matter,material_paper,material_papier mache,material_photo emulsion,material_photo material,material_photo paper,material_photo plate,material_photographic material,material_plastic,material_plastic mass,material_plywood,material_porcelain,material_printing ink,material_quartz,material_rubber,material_salt paper,material_silk,material_silver,material_silver gelatin emulsion,material_silver gelatin paper,material_skin,material_slate,material_stone,material_synthetic fibers,material_synthetic material,material_tempera,material_textile,material_tin,material_trillion,material_watercolor paint,material_wax,material_white metal,material_wire,material_wood,material_wood material,material_wooden board,material_wool,material_yarn,technique_(close/together) sewing,technique_addition,technique_aquatint,technique_ballpoint pen,technique_binding techniques,technique_black and white photo,technique_black and white photography,technique_bronzing,technique_brushing,technique_burning,technique_carving,technique_casting,technique_chalk,technique_charcoal,technique_chromogen procedure,technique_collage,technique_collotype,technique_color photo,technique_color photography,technique_colored chalk,technique_colored pencil,technique_coloring,technique_copper engraving,technique_copying,technique_crayon drawing,technique_croaking,technique_crocheting,technique_cutting,technique_cuttlefish,technique_daguerreotype,technique_digital photography,technique_digital printing,technique_drawing,technique_electronic imaging,technique_embroidery,technique_enamelling,technique_engraving,technique_etching,technique_felt tip pen,technique_glass technology,technique_gouache,technique_graphics,technique_graphite,technique_handicraft,technique_handwriting,technique_ink,technique_ink drawing,technique_intarsia,technique_kiln ceramics,technique_knitting,technique_letterpress with raster cliché,technique_linocut,technique_lithography,technique_manuscript,technique_marker,technique_mascara,technique_mezzotinto,technique_mixed media,technique_modeling,technique_molding,technique_monotypy,technique_nan,technique_offset printing,technique_oil,technique_oil painting,technique_painting,technique_painting techniques,technique_pannotype

Check the attrubuites

In [9]:
data.columns

Index(['ks', 'event_type', 'location', 'before_Christ', 'participants_role',
       'participant', 'parish', 'class', 'museum_abbr', 'musealia_mark',
       'musealia_seria_nr', 'musealia_queue_nr', 'musealia_additional_nr',
       'collection_mark', 'collection_queue_nr', 'collection_additional_nr',
       'element_count', 'is_original', 'damages', 'state', 'color', 'type',
       'source', 'city_municipality', 'country', 'material_Polish',
       'material_RC Photo Paper', 'material_a pearl',
       'material_acetate cellulose film', 'material_albumen paper',
       'material_albumin paper', 'material_aluminium', 'material_amber',
       'material_artificial fiber material', 'material_artificial leather',
       'material_artificial material', 'material_atlas',
       'material_ballpoint pen ink', 'material_birch', 'material_bone',
       'material_brass', 'material_brocade (clothing variety)',
       'material_bronze', 'material_canvas', 'material_canvas (type of cloth)',
       'ma

In [11]:
data['country'].value_counts()
#Need to add threshold of around 10?? - 19 values with value counts between 2-9, about 23 unique entries 
#Eesti = 4257 so not specific! 

Eesti                   4257
Saksamaa                 172
Vene                     170
NSVL                     143
Rootsi                    68
Soome                     45
Inglismaa                 42
Läti                      41
Ameerika Ühendriigid      38
Venemaa                   26
Prantsusmaa               16
Austria                   14
Kasahstan                 12
Kanada                    11
Itaalia                    9
Leedu                      9
Araabia                    8
Skandinaavia               8
Poola                      6
Šveits                     5
Ukraina                    4
Austraalia                 4
Taani                      4
Ecuador                    3
Ungari                     3
Jaapan                     3
Tshehhi                    3
Moldaavia                  3
Siber                      2
India                      2
Valgevene                  2
Rumeenia                   2
Liivimaa                   2
Bulgaaria                  1
Abhaasia      

In [13]:
data['city_municipality'].value_counts()
#Similar to country - maybe set thresehold to 10 
# 48 . with value_counts between 2 - 9 + alot of unique values 

Tallinn                1320
Tartu                   185
Pärnu                    83
Viljandi                 44
Haapsalu                 33
Narva                    29
Kõmsi                    22
Kohtla-Järve             18
Kesklinna                17
Rakvere                  17
Harju                    15
Narva-Jõesuu             11
Kuressaare               10
Ida-Viru                  9
Otepää                    8
Valga                     7
Setomaa vald              6
Võru                      6
Tartumaa                  6
Paide                     5
Aegviidu                  5
Tapa                      4
Pärnumaa                  4
Keila-Joa                 4
Tootsi                    4
Käsmu                     4
Sillamäe                  4
Kehtna                    3
Harjumaa Tallinn          3
Luua                      3
Lääne-Viru                3
Kaseküla                  3
Türi                      3
Martna                    3
Põlva                     3
Halliste            

In [19]:
data['event_type'].value_counts()
#Very discriptive feature! - gives a lot of clues to the type
#There are a lot of types - 551 so will need to set a sensible threshold!
#Maybe a threshold of around 30?

taking pictures                                  1376
theater performances                             1355
preparation                                      1201
theater                                          1106
culture                                           756
printing/ publishing                              431
composing                                         368
correspondence                                    341
personal and family life                          287
music                                             271
singing parties                                   257
playwriting                                       253
photography                                       196
theater activity                                  149
military and defense                              138
appearances                                       128
street                                            119
copying (making)                                  112
portrait of a man           

In [ ]:
#location_city', 'location_building', 'location_street', 'location_country', 'location_address'
#Already hot encoded? 

In [25]:
data['before_Christ'].value_counts()
#Maybe rename ei to no and then hot encode 

ei    5435
no    2391
Name: before_Christ, dtype: int64

In [46]:
data['participants_role'].value_counts()
#46 different values in total
#again quite discriptive - gives clues to the type
#some have very high value_counts - maybe set a higher threshold
#Threshold of around 100? -> 10 ok

the author              3182
liaison organization    1498
contact person           992
addressee                374
participant              318
author of the text       297
imagined                 282
publisher                262
actor                    203
photographer             196
composer                 174
the issuer               149
performer                145
the owner                139
director                 138
conductor                113
a singer                  89
compiler                  63
collector                 56
an artist                 37
user/owner                35
maker                     34
editor                    31
transferor                27
printer (maker)           25
translator                24
the arranger              23
musician                  22
a dancer                  13
engraver                  12
device author             11
co-author                 10
owner/authorizer           6
an architect               6
playwright    

In [30]:
data['parish'].value_counts()
#10/12 are unique 
#set Threshold to 2 ? 

Pärnumaa         4
Harjumaa         4
Tartumaa         1
Keila            1
Jõelähtme        1
Kuusalu          1
Jüri             1
Halliste         1
Noarootsi        1
Saaremaa         1
Rõuge            1
Pärnu-Jaagupi    1
Name: parish, dtype: int64

In [47]:
data['collection_additional_nr'].value_counts()
#4530 different values
#Set Threshold to 30? - only includes the alphabetic vlaues?  or also 100? 
# -> 10

Ar             1131
T               568
M               359
kl              355
r               341
k               248
d                81
t                43
a                31
01:01            21
03:01            19
06:01            18
14:01            18
01:02            17
01:05            15
02:01            15
02:02            14
10:01            13
01:03            13
01:04            12
T185             12
08:01            12
03:02            12
06:02            12
09:01            12
05:01            11
09:03            11
13:02            11
34:01:00         11
12:01            11
01:06            11
13:01            11
16:02            11
51:02:00         10
04:02            10
18:02            10
01.veebr         10
07:03            10
05:06            10
20:01            10
11:01            10
04:01            10
16:01            10
04:03            10
03:03            10
06:05            10
05:08             9
25:03:00          9
38:01:00          9
19:01             9


In [37]:
data['damages'].value_counts()
#786 different values
#Some features give clues such as 'The album pages are deformed, there are tears and bruises, the binding holes are broken.'
#Could set Threshold to 15 - 14 new cols -> 10
#Some columns have similar meaning - could group these eg. 'glue marks on the back', 'traces of glue on the back', 'traces of glue'

All right                                                                                                                                                                                                                                                                                                                                                                                                                                                          112
Cracked                                                                                                                                                                                                                                                                                                                                                                                                                                                            108
pin holes                                                                                 

In [40]:
data['participant'].value_counts()
#2370 - participent names
#Could either set threshold to 100 (-> 50) and have 8 new cols or have a has_value/NaN col?

J. & P. Parikas                                             307
Teater Estonia                                              284
Rahvusooper Estonia                                         221
Leichter, Karl                                              211
Tamm, Aino                                                  177
Teater Vanemuine                                            176
Alla, Armin                                                 137
Simm, Juhan                                                 107
Saarne, Henno                                                98
Teater Endla                                                 84
Leinus, Karl                                                 84
Eesti Draamateater                                           77
Franz, Erika                                                 76
Kreek, Cyrillus                                              76
Vilde, Ivar                                                  73
Visnapuu, Eduard                        

In [41]:
data['musealia_mark'].value_counts()
#can hot_encode esaily only has one value!

_    14871
Name: musealia_mark, dtype: int64

In [5]:
#data['musealia_additional_nr'].value_counts()
#a lot of unique values? - may want to add a threshold

In [6]:
#data['collection_mark'].value_counts()
#can also probably be used as is

In [7]:
#data['state'].value_counts() 
#can be hot encoded without threshold

In [8]:
#data['color'].value_counts()
#decide how to proceed with these? 
#could group by base colour first and then hot encode or use a thresehold of most pop?

## Prepping for hot encoding

### Colour grouping

In [29]:
#Grouping colours by their base colour - to avoid too many extra cloumns when hot encoding -> could always reverse this step
#by using  something like data['color'] = combined_data_translated['color'] ?

#The base colours: red, blue, green, grey, yellow, patterned, orange, brown, white, black , pink
#The most common/distingtive stay unchanged


def colour_grouping(x):
    if x in ['madara red', 'dark red', 'purple red', 'Red']:
        return 'red'
    elif x in ['light blue', 'dark blue', 'purple blue', 'greenish blue', 'greyish blue']:
        return 'blue'
    elif x in ['light green', 'light olive green', 'grey-green', 'olive green', 'greyish-olive green', 'dark green']:
        return 'green'
    elif x in ['bluish grey', 'dark grey', 'pinkish gray']:
        return 'grey'
    elif x in ['pale yellow', 'light yellow', 'orange-yellow', 'brilliant yellow']:
        return 'yellow'
    elif x in ['brownish orange']:
        return 'orange'
    elif x in ['light brown', 'dark brown', 'greyish brown', 'reddish brown', 'olive brown', 'yellowish brown']:
        return 'brown'
    elif x in ['yellowish white', 'bluish white']:
        return 'white'
    elif x in ['brownish black']:
        return 'black'
    elif x in ['mauve pink']:
        return 'pink'
    elif x in ['<patterned>', 'striped', 'checkered']:
        return 'patterned'
    else:
        return x

In [27]:
#apply colour_grouping to the dataset
data['color'] = data['color'].apply(colour_grouping)

In [28]:
#value_counts() after
data['color'].value_counts()

multicolored       153
black and white    146
black               78
white               61
blue                50
brown               26
grey                21
red                 21
yellow              17
green               17
violet              13
silver              11
golden               8
pink                 5
orange               5
transparent          4
patterned            4
monochrome           1
beige                1
Name: color, dtype: int64

### before_Christ prep

In [44]:
data['before_Christ'].value_counts()
#5435 values for 'ei' and 2391 values for 'no', ei = no
#Rename both to 'no'

ei    5435
no    2391
Name: before_Christ, dtype: int64

### Threshold 

Define a function that:
- goes throught the value_counts and selects all the values below the threshold
- labels these as 'rare and groups these together

So that during the hot_encoding we only have columns for values that appear at least 2/threshold

### Threshold Function 

In [48]:
def group_under_threshold(data, col, threshold):
    values_to_group = []
    for idx, name in enumerate(data[col].value_counts().index):
        if(data[col].value_counts()[idx] <= threshold):
            values_to_group.append(name)
    print(values_to_group)
    return data[col].apply(lambda x: 'below_thres' if (x in values_to_group) else x)
            



### Apply to columns

##### musealia_additional_nr

In [47]:
data['musealia_additional_nr'] = group_under_threshold(data, 'musealia_additional_nr', 1)

['162', '53', '75', '206', '66', '64', '31', 'c', 'a-b', '01.juuli', '77', '29', '40', '188', '139', '119', 'j', '34', '33', '221', '144', '127', '22', '145', '87', 'a-k', '41', '220', '110', '89', '57', '126', '143', '190', '125', '168', '17', '39', '241', '138', '26', 'a-i', '121', '50', 'c32', '84', '65', 'a-d', 'c19', '47', '293', '231', '72', '154', '83', '73', '185', '217', '59', '303', '132', '94', '51', '109', '151', '63', '49', '116', '105', '156', '136', '104', '113']


In [48]:
data['musealia_additional_nr'].value_counts()

1           138
R           106
rare         73
2            63
3            34
01.veebr     29
4            28
5            20
a            15
6            12
8            10
7            10
10            9
b             8
11            8
9             8
20            6
24            5
16            5
15            5
13            4
32            4
27            4
ab            4
23            4
86            3
14            3
21            3
28            3
muusika       2
197           2
37            2
18            2
12            2
62            2
19            2
36            2
a-h           2
54            2
128           2
38            2
25            2
103           2
142           2
01.apr        2
44            2
30            2
90            2
166           2
155           2
Name: musealia_additional_nr, dtype: int64

##### country

In [ ]:
#data['country'] = group_under_threshold(data, 'country', 10)

##### city_municipality

In [ ]:
#data['city_municipality'] = group_under_threshold(data, 'city_municipality', 10)

###### event type 

In [ ]:
#data['event_type'] = group_under_threshold(data, 'event_type', 30)

###### participants_role

In [ ]:
#data['participants_role'] = group_under_threshold(data, 'participants_role', 100)

###### parish

In [49]:
#Not sure it is worth it!
#data['parish'] = group_under_threshold(data, 'parish', 4)

###### collection_additional_nr

In [50]:
#data['collection_additional_nr'] = group_under_threshold(data, 'collection_additional_nr', 8)

###### damages 

In [ ]:
#may want to do a little grouping first 
#data['damages'] = group_under_threshold(data, 'damages', 16)

###### participant

In [51]:
#data['participant'] = group_under_threshold(data, 'participant', 100)

## Run hot encoding

In [49]:
columns = ['state', 'collection_mark', 'color', 'museum_abbr', 'class', 'is_orginal', 'musealia_mark']
data_dum = pd.get_dummies(data, columns = columns)

In [ ]:
data_dum.head()

In [ ]:
thres_columns = ['musealia_additional_nr', 'country', 'city_municipality', 'event_type', 'participants_role', 'parish', 'collection_additional_nr', 'damages', 'participant']
data_dum = pd.get_dummies(data_dum, columns = thres_columns)

In [50]:
data_dum.head()

,full_nr,name,ks,commentary,event_type,location,start,end,before_Christ,country_and_unit,participants_role,participant,parish,text,class,parameter,unit,value,museum_abbr,musealia_mark,musealia_seria_nr,musealia_queue_nr,collection_queue_nr,collection_additional_nr,element_count,legend,is_original,initial_info,damages,additional_text,type,source,city_municipality,country,material_Polish,material_RC Photo Paper,material_a pearl,material_acetate cellulose film,material_albumen paper,material_albumin paper,material_aluminium,material_amber,material_artificial fiber material,material_artificial leather,material_artificial material,material_atlas,material_ballpoint pen ink,material_birch,material_bone,material_brass,material_brocade (clothing variety),material_bronze,material_canvas,material_canvas (type of cloth),material_cardboard,material_cast iron,material_celluloid,material_ceramics,material_chalk,material_chamois leather,material_chamotte,material_charcoal,material_chromogen emulsion,material_chromogen paper,material_clay,material_clothing variety,material_collodion paper,material_colloid paper,material_colour,material_copper,material_cotton,material_crepe,material_crepe (cloth type),material_crystal,material_diffusion paper,material_email,material_emulsion,material_enamel paint,material_faience,material_feather,material_film,material_film (material),material_flint,material_from the bat,material_glass,material_gold,material_granite,material_graphite,material_gypsum,material_handmade paper,material_ink,material_iron,material_kalka,material_knitwear,material_leotard (type of clothing),material_linen,material_mascara,material_metal,material_metal fibers,material_moire (clothing variety),material_movie,material_nan,material_newsprint,material_nitrocellulose film,material_nut,material_oil paint,material_organic matter,material_paper,material_papier mache,material_photo emulsion,material_photo material,material_photo paper,material_photo plate,material_photographic material,material_plastic,material_plastic mass,material_plywood,material_porcelain,material_printing ink,material_quartz,material_rubber,material_salt paper,material_silk,material_silver,material_silver gelatin emulsion,material_silver gelatin paper,material_skin,material_slate,material_stone,material_synthetic fibers,material_synthetic material,material_tempera,material_textile,material_tin,material_trillion,material_watercolor paint,material_wax,material_white metal,material_wire,material_wood,material_wood material,material_wooden board,material_wool,material_yarn,technique_(close/together) sewing,technique_addition,technique_aquatint,technique_ballpoint pen,technique_binding techniques,technique_black and white photo,technique_black and white photography,technique_bronzing,technique_brushing,technique_burning,technique_carving,technique_casting,technique_chalk,technique_charcoal,technique_chromogen procedure,technique_collage,technique_collotype,technique_color photo,technique_color photography,technique_colored chalk,technique_colored pencil,technique_coloring,technique_copper engraving,technique_copying,technique_crayon drawing,technique_croaking,technique_crocheting,technique_cutting,technique_cuttlefish,technique_daguerreotype,technique_digital photography,technique_digital printing,technique_drawing,technique_electronic imaging,technique_embroidery,technique_enamelling,technique_engraving,technique_etching,technique_felt tip pen,technique_glass technology,technique_gouache,technique_graphics,technique_graphite,technique_handicraft,technique_handwriting,technique_ink,technique_ink drawing,technique_intarsia,technique_kiln ceramics,technique_knitting,technique_letterpress with raster cliché,technique_linocut,technique_lithography,technique_manuscript,technique_marker,technique_mascara,technique_mezzotinto,technique_mixed media,technique_modeling,technique_molding,technique_monotypy,technique_nan,technique_offset printing,technique_oil,technique_oil painting,techni

## Save changes to prep

In [51]:
#data_dum.to_csv('data/prep.csv')